In [1]:
### Import the data for MNIST, this will be downloaded if
### necessary or copy it over manually into the directory
### MNIST_data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Couldn't import dot_parser, loading of dot files will not be possible.


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
### Import the tensorflow module and create a session
import tensorflow as tf

### Function to initialise the weights
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

### Function to initialise the bias variable
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
### Function to quickly define the 2D convolution
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

### Function to quickly define the max pooling
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [4]:
### x is for the input y_ holds the output
x = tf.placeholder(tf.float32, shape=[None, 784]) # for the input data 28*28 image as a vector
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # output layer has 10 classes

### Define the input as 28x28 image with only 1 channel
### -1 defines that there is an unspecified number of inputs
x_image = tf.reshape(x, [-1, 28, 28, 1])

### Convolutional filter 1, 5x5 convolution with 32 "filters"
W_conv1 = weight_variable([5, 5, 1, 64])
b_conv1 = bias_variable([64])

### Convolve the input, x_image with the filter W_conv and add the bias b_conv1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # Now apply pooling

In [5]:
### Define the 2nd convolutional layer
W_conv2 = weight_variable([5, 5, 64, 128])
b_conv2 = bias_variable([128])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Define the 2nd convolutional layer
W_conv3 = weight_variable([5, 5, 128, 256])
b_conv3 = bias_variable([256])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

In [6]:
### Fully connected layer
W_fc1 = weight_variable([4 * 4 * 256, 2048])
b_fc1 = bias_variable([2048])

h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*256])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [7]:
### Make use of dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [8]:
### Define the logit/classification layer
W_fc2 = weight_variable([2048, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [9]:
### Training and evaluation functions
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
### This is the training loop
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer()) # initialise
  for i in range(4000): # go for 20000 iterations
    batch = mnist.train.next_batch(50) # with batches of 50
    if i % 200 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))    

step 0, training accuracy 0.14
step 200, training accuracy 0.96
step 400, training accuracy 1
step 600, training accuracy 0.96
step 800, training accuracy 1
step 1000, training accuracy 0.94
step 1200, training accuracy 0.98
step 1400, training accuracy 1
step 1600, training accuracy 0.94
step 1800, training accuracy 0.96
step 2000, training accuracy 0.98
step 2200, training accuracy 0.94
step 2400, training accuracy 0.98
step 2600, training accuracy 0.98
step 2800, training accuracy 1
step 3000, training accuracy 0.98
step 3200, training accuracy 0.96
step 3400, training accuracy 1
step 3600, training accuracy 0.96
step 3800, training accuracy 0.98
test accuracy 0.9826
